# Rules of survival

### Mini-project

In this small project you will use the PRISM Rule Learner algorithm to learn some rules about COVID-19 comorbidity factors. Write as much about your findings as possible. You may add external information/additional datasets for an extra-credit.

## 1. Algorithm

Copy your implementation of the correct and tested algorithm in the cell below. You do not need to supply any comments or explanations. 

In [1]:
import pandas as pd
import numpy as np

class Rule:
    def __init__(self, class_label):
        self.conditions = []  # list of conditions
        self.class_label = class_label  # rule class
        self.accuracy = 0
        self.coverage = 0

    def addCondition(self, condition):
        self.conditions.append(condition)

    def setParams(self, accuracy, coverage):
        self.accuracy = accuracy
        self.coverage = coverage
    
    # Human-readable printing of this Rule
    def __repr__(self):
        return "If {} then {}. Coverage:{}, accuracy: {}".format(self.conditions, self.class_label,
                                                                 self.coverage, self.accuracy)

class Condition:
    def __init__(self, attribute, value, true_false = None):
        self.attribute = attribute
        self.value = value
        self.true_false = true_false

    def __repr__(self):
        if self.true_false is None:
            return "{}={}".format(self.attribute, self.value)
        else:
            return "{}>={}:{}".format(self.attribute, self.value, self.true_false)

        
def refine_rule(columns, covered_subset, class_label, min_coverage=30, min_accuracy=0.6, current_rule = None, colTypes = None):
    
    # If we're refining from None, create an empty Rule
    if (current_rule == None):
        current_rule = Rule(class_label)
        

    # Some helpful variables for tracking 
    bestAcc = 0
    bestCov = 0
    bestCondition = None
    curAcc = 0
    curCov = 0 

    # We start by testing all possible columns/traits until we find the best one:
    for trait in columns:
        if trait == columns[-1]:
            continue
    
        # If the trait is numeric
        if colTypes[trait].loc['type'] == "numeric":
            # We'll split by less than and ge. 
            
            # Less than:
            
            for level in covered_subset[trait].unique().tolist():
                ruleSubset = covered_subset[covered_subset[trait] < level] # Split by less than
                
                # Make sure we don't divide by 0!
                if (len(ruleSubset) > 0):
                    curAcc = len(ruleSubset[ruleSubset[columns[-1]] == class_label]) / len(ruleSubset)
                    curCov = len(ruleSubset[ruleSubset[columns[-1]] == class_label])
                    # Evaluate effectiveness of the rule
                    if (curCov >= min_coverage):
                        if (curAcc > bestAcc):
                            bestAcc = curAcc
                            bestCov = curCov
                            bestCondition = Condition(trait, level, False)
                        if (curAcc == bestAcc):
                            if (curCov > bestCov):
                                bestCov = curCov
                                bestCondition = Condition(trait, level, False)
                
                # Now check the >= case:
                ruleSubset = covered_subset[covered_subset[trait] >= level] # Split by ge
                if (len(ruleSubset) > 0):
                    curAcc = len(ruleSubset[ruleSubset[columns[-1]] == class_label]) / len(ruleSubset)
                    curCov = len(ruleSubset[ruleSubset[columns[-1]] == class_label])
                    # Evaluate goodness of the rule
                    if (curCov >= min_coverage):
                        if (curAcc > bestAcc):
                            bestAcc = curAcc
                            bestCov = curCov
                            bestCondition = Condition(trait, level, True)
                        if (curAcc == bestAcc):
                            if (curCov > bestCov):
                                bestCov = curCov
                                bestCondition = Condition(trait, level, True)
                            
            continue # Go to the next loop
        
         
        # If our trait is categorical:
        for typeTrait in covered_subset[trait].unique().tolist():
            ruleSubset = covered_subset[covered_subset[trait] == typeTrait]
            
            # Now test for accuracy and coverage of the rule:
            curAcc = len(ruleSubset[ruleSubset[columns[-1]] == class_label]) / len(ruleSubset)
            curCov = len(ruleSubset[ruleSubset[columns[-1]] == class_label])
            if (curCov >= min_coverage):
                if (curAcc > bestAcc):
                    bestAcc = curAcc
                    bestCov = curCov
                    bestCondition = Condition(trait, typeTrait)

                # If we have another rule with the same accuracy, 
                # choose the one with the better coverage
                if (curAcc == bestAcc):
                    if curCov > bestCov:
                        bestCov = curCov
                        bestCondition = Condition(trait, typeTrait)

    # If refining does not improve our accuracy, we break
    if (bestAcc == current_rule.accuracy):
        return(None, None)
    
    # If we have no rule, break
    if (bestCondition == None):
        return(None, None)
    
    # Otherwise, add the rule to the list and
    # shorten the subset
    current_rule.addCondition(bestCondition)
    current_rule.setParams(bestAcc, bestCov)
    
    # Numeric makes things a little stranger
    if colTypes[bestCondition.attribute].loc['type'] == "numeric":
        if bestCondition.true_false:
            covered_subset = covered_subset[covered_subset[bestCondition.attribute] >= bestCondition.value]
        else:
            covered_subset = covered_subset[covered_subset[bestCondition.attribute] < bestCondition.value]
    else:
        covered_subset = covered_subset[covered_subset[bestCondition.attribute] == bestCondition.value]
    return (current_rule, covered_subset)
                            
    
    

# Ok, now we can learn_one_rule!

def learn_one_rule(columns, data, class_label, min_coverage=30, min_accuracy=0.6, colType = None):
    # Make a copy of columns so we can delete from it. 
    # Otherwise we'd be destorying columns
    columnsCopy = columns.copy().tolist()
    
    # Refine once from None, this will create the best first rule we want
    current_rule, covered_subset = refine_rule(columnsCopy, data.copy(), class_label, min_coverage, min_accuracy, None, colType)
    
    # If we got None out, it means there is no rule that meets min_coverage
    if (current_rule == None):
        return (None, None)
    
    # Remove the trait from the list of eligbile traits
    columnsCopy.remove(current_rule.conditions[-1].attribute)
    
    # We will refine while our rule isn't perfect and there are 
    # traits left to add
    while current_rule.accuracy < 1 and len(columnsCopy) > 1:
        temp_rule, temp_subset = refine_rule(columnsCopy, covered_subset,
                                             class_label, min_coverage, min_accuracy, current_rule, colType)
        # Just remove the trait we just used
        if (temp_rule != None):
            current_rule = temp_rule
            covered_subset = temp_subset
            columnsCopy.remove(current_rule.conditions[-1].attribute)
        else:
            break


    if current_rule.accuracy < min_accuracy:
        return (None, None)
    
    return (current_rule, covered_subset)
    
    # We can pre-process to determine if something is a numeric trait. 
    # We'll use two rules here:
    # 1. If the Trait is all numeric
    # 2. If there are a certain number of unique oberservations
    #    in the dataset, say more than 10. 
    # 
    # Returns a Pandas Dataframe with the columns of `columns`
    # and a row of "type" with the proper type of the variable
def learn_categories(columns, data, min_threshold = 10):
    types = []
    cols = data.columns
    for column in cols:
        if data[column].dtype == 'int64' and len(data[column].unique()) >= min_threshold:
            types.append("numeric")
        else:
            types.append("categorical")

    colType = pd.DataFrame(columns = cols)

    colType.loc['type'] = types
    return colType
    
    
def learn_rules (columns, data, classes=None, 
    min_coverage = 30, min_accuracy = 0.6):
    # List of final rules
    rules = []
    
    # If list of classes of interest is not provided - it is extracted from the last column of data
    if classes is not None:
        class_labels = classes
    else:
        class_labels = data[columns[-1]].unique().tolist()

    current_data = data.copy()

    # Get the Column types (categorical vs. numeric)
    colType = learn_categories(columns, data)

    
    
    # This follows the logic of the original PRISM algorithm
    # It processes each class in turn. 
    for class_label in class_labels:
        done = False
        while len(current_data) > min_coverage and not done:
            # Learn one rule 
            rule, subset = learn_one_rule(columns, current_data, class_label, min_coverage, min_accuracy, colType)
            
            # If the best rule does not pass the coverage threshold - we are done with this class
            if rule is None:
                break

            # If we get the rule with accuracy and coverage above threshold
            
            if rule.accuracy >= min_accuracy:
                rules.append(rule)

                # We're going to say that `subset` is the rows that need to be removed. 
                # We can drop by matching up the indices
                current_data = current_data.drop(index=subset.index)
                   
            else:
                done = True         
                
    return rules

## 2. Titanic dataset: the rules of survival

Our very familiar Titanic [dataset](https://docs.google.com/spreadsheets/d/1QGNxqRU02eAvTGih1t0cErB5R05mdOdUBgJZACGcuvs/edit?usp=sharing).

In [4]:
data_file = "../datasets/titanic.csv"

In [5]:
data = pd.read_csv(data_file)

# take a subset of attributes
data = data[['Pclass', 'Sex', 'Age', 'Survived']]

# drop all columns and rows with missing values
data = data.dropna(how="any")
print("Total rows", len(data))

column_list = data.columns.to_numpy()
print("Columns:", column_list)

Total rows 714
Columns: ['Pclass' 'Sex' 'Age' 'Survived']


In [6]:
# we can set different accuracy thresholds
# here we can reorder class labels - to first learn the rules with class label "survived".
rules = learn_rules(column_list, data, [1,0], 30, 0.7)
for rule in rules[:10]:
    print(rule)

If [Sex=female, Pclass=1] then 1. Coverage:82, accuracy: 0.9647058823529412
If [Sex=female, Pclass=2] then 1. Coverage:68, accuracy: 0.918918918918919
If [Pclass=2] then 0. Coverage:84, accuracy: 0.8484848484848485
If [Sex=male, Pclass=3] then 0. Coverage:215, accuracy: 0.849802371541502


## 3. Coronavirus: symptoms and outcome

Coronavirus [dataset](https://drive.google.com/file/d/1uVd09ekR1ArLrA8qN-Xtu4l-FFbmetVy/view?usp=sharing) (preprocessed as outlined [here](rules_motivation.ipynb)).

In [7]:
data_file = "../datasets/covid_categorical_good.csv"

In [36]:
data = pd.read_csv(data_file)
data = data.dropna(how="any")
len(data)

219179

Most accurate rules will have class label "alive". There could be too many rules, and we might never get to the class label "dead" if we rank them by accuracy. 

If we want to see which combination of attributes leads to "dead", we might want to run the algorithm with only this class label and set the lower accuracy threshold.

Remove the _age_ attribute and run your algorithm with parameters shown below.

In [10]:
# We really want to learn first what makes covid deadly
class_labels = ["dead"]
data_categorical = data.drop(columns = ['age'])
column_list = data_categorical.columns
rules = learn_rules(column_list, data_categorical, class_labels, 30, 0.6)

for rule in rules[:20]:
    print(rule)



If [renal_chronic=yes, diabetes=yes, cardiovascular=yes, obesity=no, sex=male, imm_supr=no, hypertension=yes, asthma=no, tobacco=no, copd=no] then dead. Coverage:36, accuracy: 0.6206896551724138
If [renal_chronic=yes, diabetes=yes, obesity=no, copd=yes, asthma=no, hypertension=yes, imm_supr=no, tobacco=no] then dead. Coverage:32, accuracy: 0.6274509803921569


Now try on both classes and for the entire dataset including _age_. Collect top 20 most accurate rules.

In [11]:
# This may take some time to run (took 12 min on my computer - what about your implementation?)
age_column_list = data.columns
rules = learn_rules (age_column_list, data, None, 30, 0.9)
for rule in rules[:20]:
    print(rule)

If [age>=26:False, tobacco=yes, asthma=yes] then alive. Coverage:47, accuracy: 1.0
If [age>=26:False, tobacco=yes, sex=female, obesity=yes] then alive. Coverage:83, accuracy: 1.0
If [age>=26:False, tobacco=yes, obesity=no, hypertension=no, sex=female] then alive. Coverage:273, accuracy: 0.9963503649635036
If [age>=26:False, hypertension=no, tobacco=yes, obesity=no, renal_chronic=no, imm_supr=no] then alive. Coverage:681, accuracy: 0.9970717423133236
If [age>=29:False, hypertension=no, sex=female, tobacco=yes, imm_supr=no] then alive. Coverage:331, accuracy: 1.0
If [age>=26:False, asthma=yes, obesity=no, sex=female] then alive. Coverage:246, accuracy: 1.0
If [age>=26:False, hypertension=no, sex=female, imm_supr=no, obesity=no, diabetes=no, renal_chronic=no, cardiovascular=no] then alive. Coverage:7734, accuracy: 0.9949826321883443
If [age>=30:False, hypertension=no, obesity=no, sex=female, imm_supr=no, tobacco=yes] then alive. Coverage:96, accuracy: 1.0
If [age>=30:False, hypertension=n



## 4. Discussion

### Write here a discussion about the rules that you have learned from both datasets. 

It took 7 and a half minutes to run on my computer!

### Did any of these rules surprise you?

The Titantic rules weren't that surprising, it makes sense that women in the upper classes would survive at high rates, and that's what we see in the rules. 

For the COVID data, we see that the best pridicator of death involves rules like (asthma = no, tobacoo = no, and copd = no) then dead. While that rule had pretty low accuracy, we would expect the opposite conditions to make a person more likely to die. A lot of rules for 'alive' also have some weird traits, like "tobacco = yes" or obesity = yes, etc. We likely expect worse health outcomes for smokers and obese people. 

### Do you have a meaningful logical explanation for these rules?

For the Titantic ones, definitely. I remember learning when I was young that women and children were prioritized to get on the lifeboats, and then it makes sense that people physically higher on the decks would be more likely to live. 

Hmm, not sure if it's the most meaningful, but I have a potential explanation. Suppose that when COVID patients are taken into a hospital, the hospital screens for certain characteristics that would make a person more vulnerable. If a person had weaker lungs going into the hospital, it's possible they received better/more intensive treatment than patients who had stronger lungs or other health characteristics. 

Another possible explanation is that the universe of people who smoke is pretty small. If there are a large number of non-smokers in the data, then its possible that with a disease with an already pretty low fatality rate just happened to not kill many smokers. 

### What additional research is needed to understand the meaning of your findings?

It would probably have been good to have someone actually in the hospital recording how people were processed and triaged. It would also be nice if the data were a little cleaner, this dataset wasn't designed with COVID in mind, so its possible some people didn't survive and didn't have covid at all. 

An expert on the Titnatic's history and the way it was structued, laid out, and how the lifeboats were setup, would likely have a more in-depth explanation for the rules, but I think I got the gist of it. 

## 5. Bucketing Age:

I'm curious to see how changing the ages to be bracketed changes our rules. 

In [37]:
data['age_bucket'] = "0"
data['age_bucket'] = np.where(data["age"] <= 18, "<=18", data['age_bucket'])
data['age_bucket'] = np.where((data["age"] <= 30) & (data["age"] > 18), "<=30", data['age_bucket'])
data['age_bucket'] = np.where((data["age"] <= 45) & (data["age"] > 30), "<=45", data['age_bucket'])
data['age_bucket'] = np.where((data["age"] <= 60) & (data["age"] > 45), "<=60", data['age_bucket'])
data['age_bucket'] = np.where((data["age"] > 60), ">60", data['age_bucket'])

temp_cols = data.columns
temp_cols = temp_cols.tolist()
temp_cols[-1] = temp_cols[-2]
temp_cols[-2] = 'age_bucket'
# Reorder columns now:
data = data.reindex(columns=temp_cols)
data.columns

Index(['sex', 'age', 'diabetes', 'copd', 'asthma', 'imm_supr', 'hypertension',
       'cardiovascular', 'obesity', 'renal_chronic', 'tobacco', 'age_bucket',
       'outcome'],
      dtype='object')

In [38]:
# We still drop age. 
data_categorical = data.drop(columns = ['age'])
column_list = data_categorical.columns
rules = learn_rules(column_list, data_categorical, None, 30, 0.9)

for rule in rules[:20]:
    print(rule)

If [age_bucket=<=30, sex=female, hypertension=no, tobacco=yes, imm_supr=no, diabetes=no, obesity=yes] then alive. Coverage:200, accuracy: 1.0
If [age_bucket=<=30, sex=female, hypertension=no, tobacco=yes, imm_supr=no, obesity=no, asthma=yes] then alive. Coverage:35, accuracy: 1.0
If [age_bucket=<=30, sex=female, hypertension=no, obesity=no, tobacco=yes, imm_supr=no] then alive. Coverage:668, accuracy: 0.9970149253731343
If [age_bucket=<=30, obesity=no, sex=female, asthma=yes, tobacco=no, copd=no, imm_supr=no, cardiovascular=no, renal_chronic=no, diabetes=no, hypertension=no] then alive. Coverage:404, accuracy: 0.9950738916256158
If [age_bucket=<=30, obesity=no, sex=female, asthma=yes] then alive. Coverage:37, accuracy: 1.0
If [age_bucket=<=30, obesity=no, sex=female, hypertension=no, imm_supr=no, diabetes=no, renal_chronic=no, cardiovascular=no] then alive. Coverage:13007, accuracy: 0.9960943482922346
If [age_bucket=<=18, asthma=yes] then alive. Coverage:210, accuracy: 1.0
If [age_buck

We see some fairly similar rules to what we had before, with our rules suggesting that younger people (people under 30 or 18) survive at fairly high rates. We have some funny rules, like "Age <= 18, asthma = yes" then alive or "age <= 18, tobacoo = yes" then alive. Qualitatively, these results aren't that different from treating age as a continuous variable. 

Copyright &copy; 2022 Marina Barsky. All rights reserved.